In [109]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import io as sio
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from torch.utils.data import TensorDataset, DataLoader, random_split





# 读取数据
vib_data_fault1 =  sio.loadmat('D:\Desktop\shujuronghe_20230316/vib_fault1.mat')['vib_fault1'].astype(np.float32)
curr_data_fault1 = sio.loadmat('D:\Desktop\shujuronghe_20230316/curr_fault1.mat')['curr_fault1'].astype(np.float32)
vib_data_fault2 = sio.loadmat('D:\Desktop\shujuronghe_20230316/vib_fault2.mat')['vib_fault2'].astype(np.float32)
curr_data_fault2 = sio.loadmat('D:\Desktop\shujuronghe_20230316/curr_fault2.mat')['curr_fault2'].astype(np.float32)
vib_data_fault3 =sio.loadmat('D:\Desktop\shujuronghe_20230316/vib_fault3.mat')['vib_fault3'].astype(np.float32)
curr_data_fault3 =sio.loadmat('D:\Desktop\shujuronghe_20230316/curr_fault3.mat')['curr_fault3'].astype(np.float32)
vib_data_fault4 =sio.loadmat('D:\Desktop\shujuronghe_20230316/vib_fault4.mat')['vib_fault4'].astype(np.float32)
curr_data_fault4 =sio.loadmat('D:\Desktop\shujuronghe_20230316/curr_fault4.mat')['curr_fault4'].astype(np.float32)
vib_data_fault5 =  sio.loadmat('D:\Desktop\shujuronghe_20230316/vib_fault5.mat')['vib_fault5'].astype(np.float32)
curr_data_fault5 = sio.loadmat('D:\Desktop\shujuronghe_20230316/curr_fault5.mat')['curr_fault5'].astype(np.float32)
vib_data_fault6 = sio.loadmat('D:\Desktop\shujuronghe_20230316/vib_fault7.mat')['vib_fault7'].astype(np.float32)
curr_data_fault6 = sio.loadmat('D:\Desktop\shujuronghe_20230316/curr_fault7.mat')['curr_fault7'].astype(np.float32)



print(vib_data_fault4.shape)

(2912, 384)


In [110]:
#转置成（384，2912）
vib_data_fault1 =vib_data_fault1.T 
vib_data_fault2 =vib_data_fault2.T
vib_data_fault3 =vib_data_fault3.T
vib_data_fault4 =vib_data_fault4.T
vib_data_fault5 =vib_data_fault5.T 
vib_data_fault6 =vib_data_fault6.T


curr_data_fault1=curr_data_fault1.T
curr_data_fault2=curr_data_fault2.T
curr_data_fault3=curr_data_fault3.T
curr_data_fault4=curr_data_fault4.T
curr_data_fault5=curr_data_fault5.T
curr_data_fault6=curr_data_fault6.T

# 将二维数组扩展为形状为(384, 1, 2912),再合并数组为（384，2，2912）
vib_data_fault1= np.expand_dims(vib_data_fault1, axis=1)
vib_data_fault2= np.expand_dims(vib_data_fault2, axis=1)
vib_data_fault3= np.expand_dims(vib_data_fault3, axis=1)
vib_data_fault4= np.expand_dims(vib_data_fault4, axis=1)
vib_data_fault5= np.expand_dims(vib_data_fault5, axis=1)
vib_data_fault6= np.expand_dims(vib_data_fault6, axis=1)

curr_data_fault1=np.expand_dims(curr_data_fault1, axis=1)
curr_data_fault2=np.expand_dims(curr_data_fault2, axis=1)
curr_data_fault3=np.expand_dims(curr_data_fault3, axis=1)
curr_data_fault4=np.expand_dims(curr_data_fault4, axis=1)
curr_data_fault5=np.expand_dims(curr_data_fault5, axis=1)
curr_data_fault6=np.expand_dims(curr_data_fault6, axis=1)


#合并振动和电流
data_fault1 = np.concatenate((vib_data_fault1, curr_data_fault1), axis=1)
data_fault2 = np.concatenate((vib_data_fault2, curr_data_fault2), axis=1)
data_fault3 = np.concatenate((vib_data_fault3, curr_data_fault3), axis=1)
data_fault4 = np.concatenate((vib_data_fault4, curr_data_fault4), axis=1)
data_fault5 = np.concatenate((vib_data_fault5, curr_data_fault5), axis=1)
data_fault6 = np.concatenate((vib_data_fault6, curr_data_fault6), axis=1)
# 将数据转换为 PyTorch 张量
data_fault1 = torch.tensor(data_fault1, dtype=torch.float32)
data_fault2 = torch.tensor(data_fault2, dtype=torch.float32)
data_fault3 = torch.tensor(data_fault3, dtype=torch.float32)
data_fault4 = torch.tensor(data_fault4, dtype=torch.float32)
data_fault5 = torch.tensor(data_fault5, dtype=torch.float32)
data_fault6 = torch.tensor(data_fault6, dtype=torch.float32)

print(data_fault4.shape)

torch.Size([384, 2, 2912])


一些说明：
数据集的划分通常只在训练开始前进行一次
每个epoch的minibatch都会会被重新打乱
训练集需要被打乱，测试集不需要

In [111]:

# 合并数据和标签为一个TensorDataset对象
    #torch.cat张量拼接,默认按照第一个维度拼接，如果是第二个维度可以dim=1
    #torch.tensor创建张量
dataset = TensorDataset(torch.cat((data_fault1, data_fault2, data_fault3,data_fault4,data_fault5, data_fault6)), 
                        torch.tensor([0]*384 + [1]*384 + [2]*384 +[3]*384 +[4]*384 +[5]*384))

# 划分训练集和测试集
    #random_split是生成随机数再进行划分 随机的
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 创建训练集和测试集的DataLoader对象
    #batch是一次使用小批量数据 计算平均梯度来更新参数
    #每一个epoch都会打乱数据
train_dataloader = DataLoader(train_dataset, batch_size=90, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=90, shuffle=False)

# 打印训练集和测试集的样本数
print("Number of training samples: ", len(train_dataset))
print("Number of testing samples: ", len(test_dataset))


Number of training samples:  1612
Number of testing samples:  692


以下定义了我的卷积神经网络，其中super是继承了nn.module类。
对于卷积层的说明：有几个卷积核就有几个输出通道，1个卷积核一次是对于2个输入通道同时进行卷积运算提取特征，也就是说会一次处理两层特征曾。每一个卷积核都在初始化阶段 随机初始化权重，因而每个卷积核开始时学习的“起点”不同，虽然拥有相同的结构，但依然会提取到不同的特征。
对于池化层的说明，这里使用了一维池化层，所谓一维是对每个通道独立进行，每次池化后数据长度减半。
对于全连接层的说明：

In [112]:
class FusionNet(nn.Module):
    def __init__(self, hidden_size):
        super(FusionNet, self).__init__() #super是为了继承自nn.module类，调用了父类的初始化方法
        self.conv1 = nn.Conv1d(2, 32, kernel_size=3, padding=1)  # 第一层卷积层（输入通道数，输出通道数，卷积核大小，左右各补1个0）
        self.relu1 = nn.ReLU()  # 激活函数
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)  # 最大池化层（池化窗口大小，滑动步幅）
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)  # 第二层卷积层
        self.relu2 = nn.ReLU()  # 激活函数
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)  # 最大池化层
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)  # 第三层卷积层
        self.relu3 = nn.ReLU()  # 激活函数
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)  # 最大池化层
        self.conv4 = nn.Conv1d(128, 256, kernel_size=3, padding=1)  # 第四层卷积层
        self.relu4 = nn.ReLU()  # 激活函数
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)  # 最大池化层
        self.fc1 = nn.Linear(256 * 182, hidden_size)  # 全连接层（通道数*池化层输出的长度，输出层的维度）
        self.dropout = nn.Dropout(0.5)  # Dropout层
        
    def forward(self, x): #这里要注意连接的顺序，init里定义时则不需要
        x = self.pool1(self.relu1(self.conv1(x)))  # 第一层卷积层的前向传播
        x = self.pool2(self.relu2(self.conv2(x)))  # 第二层卷积层的前向传播
        x = self.pool3(self.relu3(self.conv3(x)))  # 第三层卷积层的前向传播
        x = self.pool4(self.relu4(self.conv4(x)))  # 第四层卷积层的前向传播
        x = x.view(x.shape[0], -1)  # 将卷积层的输出展平为一维张量
        x = self.dropout(x)  # Dropout层的前向传播
        x = self.fc1(x)  # 全连接层的前向传播
        return x


没有实例化，没有给input_size等赋值

In [113]:
#分类网络
class Net2(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) #全连接层
        self.relu = nn.ReLU()  #激活层
        self.fc2 = nn.Linear(hidden_size, num_classes) #全连接层
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x



In [114]:
hidden_size = 1000
fusion_net = FusionNet(hidden_size=1000)
#input_size是通过查找fusion_net的输出特征数（fusion_net.fc1.out_features）获得的
net2 = Net2(input_size=fusion_net.fc1.out_features, hidden_size=500, num_classes=6)
model = nn.Sequential(fusion_net, net2)
#模型的输入是fusion_net的输入，即三通道的信号数据。经过fusion_net的卷积、池化和全连接层之后，得到一个特征向量，。然后，这个特征向量会被传递到net2，net2会将其映射到一个长度为num_classes的向量，表示三个类别的概率分布



In [115]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


这个函数定义没有self，是一个独立的训练函数但不是类的方法

In [116]:
def train(model, dataloader, criterion, optimizer):
    model.train()     #将模型调整到训练模式
    running_loss = 0.0    #每个mini-batch的平均损失
    running_corrects = 0  #当前迭代过程中预测正确的样本数
    total_samples = 0     #当前迭代过程中总的样本数
    for i, data in enumerate(dataloader, 0): #遍历dataloader中的每个minibatch 起始索引 0
        inputs, labels = data           #讲每个mini-batch分解为两个变量
        optimizer.zero_grad()      #清除梯度缓存
        outputs = model(inputs)   #模型的输出
        loss = criterion(outputs, labels)     #损失值 criterion计算
        loss.backward()    #计算loss的梯度
        optimizer.step()   #更新模型参数
        running_loss += loss.item() #累加每个mini-batch的loss，为了之后计算整个epoch的loss
        _, preds = torch.max(outputs, 1) #模型输出的每个样本在各类别上的分数，然后将分数最高的类别作为该样本的预测结果
        running_corrects += torch.sum(preds == labels.data) #当前迭代过程中预测正确的样本数。
        total_samples += labels.size(0) #当前迭代过程中的总样本数。
    acc = running_corrects.double() / total_samples
    return running_loss / len(dataloader), acc  #每个mini-batch的平均损失和预测准确率

def test(model, dataloader):
    model.eval() #切换到评估模式
    correct = 0 #初始化正确的预测数
    total = 0  #初始化总样本
    with torch.no_grad(): #关闭pytorch的梯度追踪
        for data in dataloader:
            inputs, labels = data  #输入数据和标签
            outputs = model(inputs)  #模型的输出
            _, predicted = torch.max(outputs.data, 1) #取概率最高的作为预测结果，1表示在维度1上取最大值
            #返回1最大值的张量，2对应索引的张量，_表示我们不关心第一个只需要第二个，也就是样本预测类别索引
            total += labels.size(0) #累加当前batch中的样本数到总正确预测数中
            #(predicted == labels)是布尔张量，结果为True/False，求和，取python标量值
            correct += (predicted == labels).sum().item() #累加当前batch中正确的预测样本数 到总正确预测数中
    return correct / total





In [ ]:
# 训练模型
num_epochs =170
train_loss = []
train_acc = []
test_acc = []
for epoch in range(num_epochs):
    loss, acc = train(model, train_dataloader, criterion, optimizer)
    train_loss.append(loss)
    train_acc.append(acc)
    test_acc.append(test(model, test_dataloader))
    print('Epoch: %d, Train Loss: %.3f, Train Accuracy: %.3f, Test Accuracy: %.3f' %
          (epoch+1, loss, acc, test_acc[-1]))
    # 每个epoch结束后绘制图像
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.plot(train_loss)
plt.xlabel('Epochs')
plt.ylabel('Train Loss')
plt.title('Training Loss')

plt.subplot(1, 2, 2)
plt.plot(test_acc, label='Test Accuracy')
plt.plot(train_acc, label='Train Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy')
plt.legend()

plt.show()


Epoch: 1, Train Loss: 1.792, Train Accuracy: 0.167, Test Accuracy: 0.143
Epoch: 2, Train Loss: 1.781, Train Accuracy: 0.231, Test Accuracy: 0.324
Epoch: 3, Train Loss: 1.703, Train Accuracy: 0.311, Test Accuracy: 0.324
Epoch: 4, Train Loss: 1.656, Train Accuracy: 0.335, Test Accuracy: 0.342
Epoch: 5, Train Loss: 1.651, Train Accuracy: 0.334, Test Accuracy: 0.488
Epoch: 6, Train Loss: 1.642, Train Accuracy: 0.364, Test Accuracy: 0.324
Epoch: 7, Train Loss: 1.642, Train Accuracy: 0.346, Test Accuracy: 0.421
Epoch: 8, Train Loss: 1.642, Train Accuracy: 0.367, Test Accuracy: 0.361
Epoch: 9, Train Loss: 1.639, Train Accuracy: 0.362, Test Accuracy: 0.361
Epoch: 10, Train Loss: 1.637, Train Accuracy: 0.364, Test Accuracy: 0.490
Epoch: 11, Train Loss: 1.629, Train Accuracy: 0.385, Test Accuracy: 0.408
Epoch: 12, Train Loss: 1.630, Train Accuracy: 0.380, Test Accuracy: 0.366
Epoch: 13, Train Loss: 1.632, Train Accuracy: 0.373, Test Accuracy: 0.324
Epoch: 14, Train Loss: 1.637, Train Accuracy: 0

In [ ]:
def evaluate(model, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

test_acc = evaluate(model, test_dataloader)
print('Test Accuracy: %.3f' % test_acc)


In [1]:
import os
print(os.path.abspath('.'))


C:\Users\lenovo\数据融合
